1. Conectarme a la base de datos

In [ ]:
from os import write
#Librerías necesarias para trabajar con la base de datos
import pandas as pd
from sqlalchemy import create_engine
import sqlite3

#Conectamos nuestra DB
engine = create_engine('sqlite:///empresa2.db')

#crear una función que me permita conectarme a esa base de datos y hacer las consultas que yo desee
from sqlalchemy import text

def run_query(query: str):
    with engine.connect() as conn:
        # Check if the query is a modifying query
        if query.strip().lower().startswith(('insert', 'update', 'delete')):
            conn.execute(text(query)) # Execute queries as text
            conn.commit() # Commit the transaction
            return "Query executed successfully."
        else:
            # For read-only queries
            result = pd.read_sql_query(query, conn)
            return result


2. Consultas sencilla

In [ ]:
query1="""
    SELECT * FROM empleados
"""
result1 = run_query(query1)
print("Consulta sencilla")
result1

Consulta sencilla


,Id,nombre,apellido,documento,departamento,manager,salario,experiencia
0,100,Carlos,Matta,8325476,CEO,0,5000000,Senior
1,101,Daniela,Gutierrez,28564789,Dirección Comercial,100,3800000,Semisenior
2,102,Cesar,Jiménez,1025453211,Dirección Financiera,100,3800000,Junior
3,103,Camila,Quiroz,1037412587,Dirección Marketing,100,3800000,Junior
4,104,Anderson,Benjumea,1037413698,Direccion de Talento Humano,101,2500000,Senior
5,105,Catalina,Castaño,26784513,Asesoría Comercial,101,1800000,Senior
6,106,Esteban,Cifuentes,1110852697,Contaduría,102,2500000,Semisenior
7,107,Mariana,Restrepo,1023458751,Tesorería,102,2500000,Junior
8,108,Andrea,López,1024785412,Marketing Digital,103,2500000,Junior
9,109,Tomás,Flórez,54123547,Publicidad,103,3200000,Senior


2. Subconsulta 1

Clasificar las filas basadas en un criterio de order

In [ ]:
query2="""
    SELECT
    Id,
    apellido,
    nombre,
    salario,
    RANK() OVER (ORDER BY salario DESC) AS ranking
    FROM empleados
    ORDER BY ranking
"""
result2 = run_query(query2)
print("Subconsulta 2")
result2


Subconsulta 2


,Id,apellido,nombre,salario,ranking
0,100,Matta,Carlos,5000000,1
1,101,Gutierrez,Daniela,3800000,2
2,102,Jiménez,Cesar,3800000,2
3,103,Quiroz,Camila,3800000,2
4,109,Flórez,Tomás,3200000,5
5,104,Benjumea,Anderson,2500000,6
6,106,Cifuentes,Esteban,2500000,6
7,107,Restrepo,Mariana,2500000,6
8,108,López,Andrea,2500000,6
9,105,Castaño,Catalina,1800000,10


3. Ejemplo 3: Mostrar las primeras 5 filas de un conjunto de resultados

Através de un CTE

In [ ]:
query3 = """
    WITH empleados_rankeados AS (
        SELECT
            Id,
            apellido,
            nombre,
            salario,
            RANK() OVER (ORDER BY salario DESC) AS ranking
        FROM empleados
        )
    SELECT
        Id,
        apellido,
        nombre,
        salario
        FROM empleados_rankeados
        WHERE ranking <= 5
        ORDER BY ranking
"""
result3 = run_query(query3)
print("Ejemplo 3")
result3


Ejemplo 3


,Id,apellido,nombre,salario
0,100,Matta,Carlos,5000000
1,101,Gutierrez,Daniela,3800000
2,102,Jiménez,Cesar,3800000
3,103,Quiroz,Camila,3800000
4,109,Flórez,Tomás,3200000


4. Ejemplo 4

Listar las últimas 5 filas de un conjunto de datos

In [ ]:
query4 = """
    WITH empleados_rankeados AS (
        SELECT
            Id,
            apellido,
            nombre,
            salario,
            RANK() OVER (ORDER BY salario DESC) AS ranking
        FROM empleados
        )
    SELECT
        Id,
        apellido,
        nombre,
        salario
        FROM empleados_rankeados
        WHERE ranking <= 5
        ORDER BY ranking
"""
result4 = run_query(query4)
print("Ejemplo 4")
result4

Ejemplo 4


,Id,apellido,nombre,salario
0,100,Matta,Carlos,5000000
1,101,Gutierrez,Daniela,3800000
2,102,Jiménez,Cesar,3800000
3,103,Quiroz,Camila,3800000
4,109,Flórez,Tomás,3200000


5. Listar la segunda fila más alta de los datos consultados.

otra manera=> ROW_NUMBER() OVER (ORDER BY salario DESC) AS ranking

In [ ]:
query5 = """
    WITH empleados_rankeados AS (
        SELECT
            Id,
            apellido,
            nombre,
            salario,
            DENSE_RANK() OVER (ORDER BY salario DESC) AS ranking
        FROM empleados
        )
    SELECT
        Id,
        apellido,
        nombre,
        salario
        FROM empleados_rankeados
        WHERE ranking =2
        ORDER BY ranking
"""
result5 = run_query(query5)
print("Ejemplo 5")
result5

Ejemplo 5


,Id,apellido,nombre,salario
0,101,Gutierrez,Daniela,3800000
1,102,Jiménez,Cesar,3800000
2,103,Quiroz,Camila,3800000


7. Mostremos la mediana del conjunto de datos consultado (50%)

In [ ]:
query7 = """
    WITH empleados_rankeados AS (
        SELECT
            Id,
            apellido,
            nombre,
            salario,
            NTILE(2) OVER (ORDER BY salario ASC) AS ntile
        FROM empleados
        )
    SELECT
        Id,
        apellido,
        nombre,
        salario
        FROM empleados_rankeados
        WHERE ntile = 1
        ORDER BY salario
"""
result7 = run_query(query7)
print("Ejemplo 7")
result7

Ejemplo 7


,Id,apellido,nombre,salario
0,105,Castaño,Catalina,1800000
1,104,Benjumea,Anderson,2500000
2,106,Cifuentes,Esteban,2500000
3,107,Restrepo,Mariana,2500000
4,108,López,Andrea,2500000


7. Ejemplo 7: Vamos a listar el último 20% de los registros del conjunto de datos.

In [ ]:
query7 = """
    WITH empleados_rankeados AS (
        SELECT
            Id,
            apellido,
            nombre,
            salario,
            NTILE(5) OVER (ORDER BY salario ASC) AS ntile
        FROM empleados
        )
    SELECT
        Id,
        apellido,
        nombre,
        salario
        FROM empleados_rankeados
        WHERE ntile = 5
        ORDER BY salario
"""
result7 = run_query(query7)
print("Ejemplo 7")
result7

Ejemplo 7


,Id,apellido,nombre,salario
0,103,Quiroz,Camila,3800000
1,100,Matta,Carlos,5000000


8. Ejemplo 8: Vamos a listar el entre el 40 y el 70% de los registros del conjunto de datos

In [ ]:
query8 = """
    WITH empleados_rankeados AS (
        SELECT
            Id,
            apellido,
            nombre,
            salario,
            NTILE(10) OVER (ORDER BY salario ASC) AS ntile
        FROM empleados
        )
    SELECT
        Id,
        apellido,
        nombre,
        salario
        FROM empleados_rankeados
        WHERE ntile >4 AND ntile <=7
        ORDER BY salario
"""
result8 = run_query(query8)
print("Ejemplo 8")
result8

Ejemplo 8


,Id,apellido,nombre,salario
0,108,López,Andrea,2500000
1,109,Flórez,Tomás,3200000
2,101,Gutierrez,Daniela,3800000


9. Ejmeplo 9: Mostrar todas los registros que estén por encima del promedio

In [ ]:
query9 = """
    SELECT
        nombre,
        apellido,
        salario
        FROM empleados
        WHERE salario > (SELECT AVG(salario) FROM empleados)
"""
result9 = run_query(query9)

query9_2 = """
    SELECT AVG(salario) FROM empleados
"""
result9_2 = run_query(query9_2)


print("Ejemplo 9\n")
print("Promedio de salarios\n")
print(result9_2)
print("\nSalarios por encima del promedio")
result9

Ejemplo 9

Promedio de salarios

   AVG(salario)
0     3140000.0

Salarios por encima del promedio


,nombre,apellido,salario
0,Carlos,Matta,5000000
1,Daniela,Gutierrez,3800000
2,Cesar,Jiménez,3800000
3,Camila,Quiroz,3800000
4,Tomás,Flórez,3200000


In [ ]:
query10 = """
    SELECT
        nombre,
        apellido,
        salario
        FROM empleados
        WHERE salario < (SELECT AVG(salario) FROM empleados)
"""
result10 = run_query(query10)

query10_2 = """
    SELECT AVG(salario) FROM empleados
"""
result10_2 = run_query(query10_2)


print("Ejemplo 10\n")
print("Promedio de salarios\n")
print(result10_2)
print("\nSalarios por debajo del promedio")
result10

Ejemplo 10

Promedio de salarios

   AVG(salario)
0     3140000.0

Salarios por debajo del promedio


,nombre,apellido,salario
0,Anderson,Benjumea,2500000
1,Catalina,Castaño,1800000
2,Esteban,Cifuentes,2500000
3,Mariana,Restrepo,2500000
4,Andrea,López,2500000


11. Ejemplo 11: Obtener todos los registros donde un valor este relacionado con una subconsulta; Para este ejemplo usemos la columna de jefe inmediato, para mostrar, cuáles personas son empleados de determinado jefe de departamento.

In [ ]:
query11 = """
    SELECT
        nombre,
        apellido
        FROM empleados
        WHERE Id IN (
            SELECT Id
            FROM empleados
            WHERE manager = 100
        )
"""

query11_2 = """
    SELECT
        nombre,
        apellido
        FROM empleados
        WHERE Id = 100
"""

result11 = run_query(query11)
print("Ejemplo 11")
result11_2 = run_query(query11_2)

print(result11_2)

print("\nEmpleados del jefe 101")


result11

Ejemplo 11
   nombre apellido
0  Carlos    Matta

Empleados del jefe 101


,nombre,apellido
0,Daniela,Gutierrez
1,Cesar,Jiménez
2,Camila,Quiroz


12. Ejemplo 12: Unir una tabla a sí misma; mostrar el jefe inmediato de cada empleado

In [ ]:
query12 = """
    SELECT
        e1.nombre || ' ' || e1.apellido AS manager_nombre,
        e2.nombre || ' ' || e2.apellido AS empleado_nombre
        FROM empleados e1
        JOIN empleados e2
        ON e1.Id = e2.manager
        WHERE e1.Id = 102
"""

result12 = run_query(query12)
print("Ejemplo 12")
result12

Ejemplo 12


,manager_nombre,empleado_nombre
0,Cesar Jiménez,Esteban Cifuentes
1,Cesar Jiménez,Mariana Restrepo


In [ ]:
#comentario
#voy a crear una variable string
variableStringCorta = "Hola"
print(variableStringCorta)
variableStringLarga = """Hola
como
estas
como
te
va"""
print(variableStringLarga)

13. Agrupación de datos - GROUP BY  - ROLLUP
Vamos a generar un informe con el importe total del salario por departamento y nivel de experiecia.
`



Otras maneras:

import sqlite3
import pandas as pd

# Establish the connection to SQLite database
db_path = 'your_database.db' # Path to your SQLite database file

# Define a function to run queries, including modifications
def run_query(query: str):
with sqlite3.connect(db_path) as conn:
# Check if the query is a modifying query
if query.strip().lower().startswith(('insert', 'update', 'delete')):
conn.execute(query) # Execute the modifying query
conn.commit() # Commit the transaction
return "Query executed successfully."
else:
# For read-only queries
result = pd.read_sql_query(query, conn)
return result

__________________________________________________________

cursorObj = conexion.cursor()
cursorObj.execute("""   
    INSERT INTO empleados (
        Id,
        nombre,
        apellido,
        documento,
        departamento,
        manager,
        salario,
        experiencia)
    VALUES (
        110,
        'Cristian',
        'Castro',
        52657877,
        'Dirección de Talento Humano',
        101,
        2720000,
        'Senior'
    )
"""
)
query_insert = """   
    INSERT INTO empleados (
        Id,
        nombre,
        apellido,
        documento,
        departamento,
        manager,
        salario,
        experiencia)
    VALUES (
        115,
        'Cristian',
        'Castro',
        52657,
        'Dirección de Talento Humano',
        101,
        2720000,
        'Senior'
    )
"""
conexion.execute(query_insert)



In [ ]:
#insertar un registro nuevo
query_insert ="""
    INSERT INTO empleados (
        Id,
        nombre,
        apellido,
        documento,
        departamento,
        manager,
        salario,
        experiencia)
    VALUES (
        111,
        'Salomé',
        'Gaviria',
        123456789,
        'Direccion de Talento Humano',
        101,
        2800000,
        'Junior'
    )
"""
result_insert = run_query(query_insert)

In [ ]:
#actualizar un registro que quedó guardado con un error

query_update = """
    UPDATE empleados
    SET documento = '5252'
    WHERE Id = 112
"""
result_update = run_query(query_update)

In [ ]:
query13 = """
    SELECT
        departamento,
        experiencia,
        SUM(salario) total_salario
        FROM empleados
        GROUP BY departamento, experiencia
"""
result13 = run_query(query13)
print("Ejemplo 13")
result13

Ejemplo 13


,departamento,experiencia,total_salario
0,Asesoría Comercial,Senior,1800000
1,CEO,Senior,5000000
2,Contaduría,Semisenior,2500000
3,Direccion de Talento Humano,Senior,8020000
4,Dirección Comercial,Semisenior,3800000
5,Dirección Financiera,Junior,3800000
6,Dirección Marketing,Junior,3800000
7,Marketing Digital,Junior,2500000
8,Publicidad,Senior,3200000
9,Tesorería,Junior,2500000


14. Ejemplo 14. Creemos una sumatoria condicional: Obtengamos el total de salarios de los departamentos de asesor comercial y recursos humanos combinados.

In [ ]:
query14 = """
    SELECT
        SUM(CASE
            WHEN departamento IN ('Asesoría Comercial','Direccion de Talento Humano')
            THEN salario
            ELSE 0
            END) AS total_salario_comercial_y_talento_humano
        FROM empleados
"""
result14 = run_query(query14)

print("Ejemplo 14\n")
print("Total de los salarios de las personas de Asesoría Comercial y Dirección de Talento Humano\n")
#Forma 1
salarios1 = result14.iloc[0,0]
print(f"\nEl total de los salarios de asesoría comercial y talento humano es {salarios1}\n")

#Forma 3
print("Total Salarios Comercial y Talento Humano:", f'$ {result14.iloc[0,0]:,.0f}',"\n")

#Forma 2
result14


Ejemplo 14

Total de los salarios de las personas de Asesoría Comercial y Dirección de Talento Humano


El total de los salarios de asesoría comercial y talento humano es 9820000

Total Salarios Comercial y Talento Humano: $ 9,820,000 



,total_salario_comercial_y_talento_humano
0,9820000


15. Ejemplo 15: Anidando sumatorias de la consulta anterior

In [ ]:
query15 = """
    SELECT
        SUM(CASE
            WHEN departamento IN ('Asesoría Comercial','Direccion de Talento Humano')
            THEN salario
            ELSE 0
            END) AS total_salario_comercial_y_talento_humano,
            SUM(CASE
            WHEN departamento IN ('Tesorería','Contaduría')
            THEN salario
            ELSE 0
            END) AS total_salario_tesoreria_y_contaduria
        FROM empleados
"""
result15 = run_query(query15)

print("Ejemplo 15\n")
print("Total de los salarios de las personas de Asesoría Comercial y Dirección de Talento Humano, Contaduría y Tesorería\n")
result15

Ejemplo 15

Total de los salarios de las personas de Asesoría Comercial y Dirección de Talento Humano, Contaduría y Tesorería



,total_salario_comercial_y_talento_humano,total_salario_tesoreria_y_contaduria
0,9820000,5000000


16. Ejemplo 16: Agrupemos filas de acuerdo a un rango salarial

In [ ]:
query16 = """
    SELECT
        CASE
            WHEN salario <= 2500000 THEN 'Nivel 1'
            WHEN salario > 2500000 AND salario <= 3800000 THEN 'Nivel 2'
            WHEN salario > 3800000 THEN 'Nivel 3'
        END AS nivel_salario,
        COUNT(*) AS cantidad_empleados
        FROM empleados
        GROUP BY
        CASE
            WHEN salario <= 2000000 THEN 'Nivel 1'
            WHEN salario > 2500000 AND salario <= 3800000 THEN 'Nivel 2'
            WHEN salario > 3800000 THEN 'Nivel 3'
        END
"""
result16 = run_query(query16)
print("Ejemplo 16")
result16

Ejemplo 16


,nivel_salario,cantidad_empleados
0,Nivel 1,4
1,Nivel 1,1
2,Nivel 2,6
3,Nivel 3,1


1hora   19
45 MIN  7
30 MIN  3